# Songnality Audio Analysis

Notes:
- Using Client Credentials Code Flow process. This uses server to server authentication which does not access user information. Therefore, a Spotipy redirect URI is not needed (the browser Spotify sign in process), and there is a higher rate limit applied.
- Songnality only uses Spotify's standard audio_features. This returns basic overall information on the tracks, compared to Spotify's audio_analysis method, which returns a more thorough low-level track analysis. Oddly, some of the information covered by both methods contrast with each other. This is a point of potential expansion to improve Songnality's personality matching and recommendation system.

In [14]:
# Import Spotify API package
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Import pandas package
import pandas as pd

## API access section

Write a blurb about API stuff here

In [ ]:
PLAYLIST_OFFSET = 100

In [2]:
# Import Spotify API information
from config import client_id, client_secret

In [6]:
# Create a Spotify API client with authorized API credentials (id and secret)
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [35]:
def get_track_info(track_uri):
    basic_info = sp.track(track_uri)
    audio_info = sp.audio_features(track_uri)[0]

    artists = []
    for artist_info in basic_info['artists']:
        artists.append({
            'name': artist_info['name'],
            'artist_link': artist_info['external_urls']['spotify']
        })

    # name, popularity, album > images, external_urls > spotify, album > release_date, album > album_type
    track_info = {
        'name': basic_info['name'],
        'artists': artists,
        'popularity': basic_info['popularity'],
        'image_link': basic_info['album']['images'][1]['url'],
        'track_link': basic_info['external_urls']['spotify'],
        'release_date': basic_info['album']['release_date'],
        'album_type': basic_info['album']['album_type']
    }

    track_info.update(audio_info)
    return track_info

In [7]:
# Set a playlist link to collect information on
playlist_link = "https://open.spotify.com/playlist/4J1tWRxG58OxCqVgUwRY9i?si=4ba873e110bc4afd"

In [50]:
# Function to loop through playlist for more than 100 tracks
def get_playlist_tracks(playlist_link):
    playlist_URI = playlist_link.split("/")[-1].split("?")[0]
    playlist_info = sp.playlist_tracks(playlist_URI, limit=100, offset=0)

    if playlist_info['total'] > 100:
        for tracks_iteration in range(1, (playlist_info['total'] // 100) + 1):
            playlist_info['items'].extend(sp.playlist_tracks(playlist_URI, offset=(100 * tracks_iteration))['items'])

    track_info = [get_track_info(track["track"]["uri"]) for track in playlist_info["items"]]
    return track_info

In [51]:
test = get_playlist_tracks(playlist_link)

7


In [52]:
test

[{'name': 'Hold Me Down - Live At The Troubadour',
  'artists': [{'name': 'The Happy Fits',
    'artist_link': 'https://open.spotify.com/artist/73rPcaYEhBd0UuVZBqqyQJ'}],
  'popularity': 11,
  'image_link': 'https://i.scdn.co/image/ab67616d00001e02b45901a48e9eeebc074bdaaa',
  'track_link': 'https://open.spotify.com/track/2doc5MI6NR7XOW5b0ZSlXP',
  'release_date': '2023-03-17',
  'album_type': 'album',
  'danceability': 0.313,
  'energy': 0.812,
  'key': 2,
  'loudness': -7.241,
  'mode': 1,
  'speechiness': 0.0393,
  'acousticness': 0.667,
  'instrumentalness': 0.000946,
  'liveness': 0.802,
  'valence': 0.692,
  'tempo': 161.471,
  'type': 'audio_features',
  'id': '2doc5MI6NR7XOW5b0ZSlXP',
  'uri': 'spotify:track:2doc5MI6NR7XOW5b0ZSlXP',
  'track_href': 'https://api.spotify.com/v1/tracks/2doc5MI6NR7XOW5b0ZSlXP',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2doc5MI6NR7XOW5b0ZSlXP',
  'duration_ms': 299816,
  'time_signature': 4},
 {'name': 'Keep Driving',
  'artists': 

In [53]:
len(test)

119

In [54]:
song_features_df = pd.DataFrame(test)

In [55]:
song_features_df

,name,artists,popularity,image_link,track_link,release_date,album_type,danceability,energy,key,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Hold Me Down - Live At The Troubadour,"[{'name': 'The Happy Fits', 'artist_link': 'ht...",11,https://i.scdn.co/image/ab67616d00001e02b45901...,https://open.spotify.com/track/2doc5MI6NR7XOW5...,2023-03-17,album,0.313,0.812,2,...,0.8020,0.692,161.471,audio_features,2doc5MI6NR7XOW5b0ZSlXP,spotify:track:2doc5MI6NR7XOW5b0ZSlXP,https://api.spotify.com/v1/tracks/2doc5MI6NR7X...,https://api.spotify.com/v1/audio-analysis/2doc...,299816,4
1,Keep Driving,"[{'name': 'Harry Styles', 'artist_link': 'http...",75,https://i.scdn.co/image/ab67616d00001e022e8ed7...,https://open.spotify.com/track/2NcQic8JxdjAlAH...,2022-05-20,album,0.721,0.479,9,...,0.1820,0.902,164.948,audio_features,2NcQic8JxdjAlAHuNbOIRE,spotify:track:2NcQic8JxdjAlAHuNbOIRE,https://api.spotify.com/v1/tracks/2NcQic8JxdjA...,https://api.spotify.com/v1/audio-analysis/2NcQ...,140242,3
2,I Love It (feat. Charli XCX),"[{'name': 'Icona Pop', 'artist_link': 'https:/...",75,https://i.scdn.co/image/ab67616d00001e024e2e60...,https://open.spotify.com/track/6HZ9VeI5IRFCNQL...,2013-08-27,album,0.711,0.906,8,...,0.1530,0.824,125.916,audio_features,6HZ9VeI5IRFCNQLXhpF4bq,spotify:track:6HZ9VeI5IRFCNQLXhpF4bq,https://api.spotify.com/v1/tracks/6HZ9VeI5IRFC...,https://api.spotify.com/v1/audio-analysis/6HZ9...,157153,4
3,"Now or Never (feat. Charlie Gillespie, Owen Pa...","[{'name': 'Julie and the Phantoms Cast', 'arti...",54,https://i.scdn.co/image/ab67616d00001e02b51d34...,https://open.spotify.com/track/2h4BV7jx9Ltz9Nx...,2020-09-10,album,0.449,0.957,2,...,0.0242,0.785,175.954,audio_features,2h4BV7jx9Ltz9NxC2zgNcW,spotify:track:2h4BV7jx9Ltz9NxC2zgNcW,https://api.spotify.com/v1/tracks/2h4BV7jx9Ltz...,https://api.spotify.com/v1/audio-analysis/2h4B...,184128,4
4,Daze Inn,"[{'name': 'Carlie Hanson', 'artist_link': 'htt...",42,https://i.scdn.co/image/ab67616d00001e027dfe86...,https://open.spotify.com/track/20L6erw0ny2hiwQ...,2020-03-27,single,0.635,0.768,0,...,0.3820,0.580,149.010,audio_features,20L6erw0ny2hiwQg8c1Dj4,spotify:track:20L6erw0ny2hiwQg8c1Dj4,https://api.spotify.com/v1/tracks/20L6erw0ny2h...,https://api.spotify.com/v1/audio-analysis/20L6...,158940,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,Slow (with Ciara),"[{'name': 'Jackson Wang', 'artist_link': 'http...",56,https://i.scdn.co/image/ab67616d00001e0221a2b6...,https://open.spotify.com/track/3ZrtBTReHr1xFDg...,2023-04-17,single,0.863,0.593,5,...,0.1210,0.557,116.063,audio_features,3ZrtBTReHr1xFDglmHwCX5,spotify:track:3ZrtBTReHr1xFDglmHwCX5,https://api.spotify.com/v1/tracks/3ZrtBTReHr1x...,https://api.spotify.com/v1/audio-analysis/3Zrt...,192182,4
115,Hot,"[{'name': 'OSTON', 'artist_link': 'https://ope...",29,https://i.scdn.co/image/ab67616d00001e02cfa532...,https://open.spotify.com/track/772eoZ8AsmYyK8F...,2023-06-28,single,0.592,0.501,1,...,0.1090,0.522,101.933,audio_features,772eoZ8AsmYyK8Fhq2qS7y,spotify:track:772eoZ8AsmYyK8Fhq2qS7y,https://api.spotify.com/v1/tracks/772eoZ8AsmYy...,https://api.spotify.com/v1/audio-analysis/772e...,193800,4
116,Superhero,"[{'name': 'Lauv', 'artist_link': 'https://open...",56,https://i.scdn.co/image/ab67616d00001e025426d8...,https://open.spotify.com/track/6mipiX3H43Lz4vI...,2018-08-30,single,0.680,0.340,7,...,0.2850,0.418,130.027,audio_features,6mipiX3H43Lz4vIJTG6cdO,spotify:track:6mipiX3H43Lz4vIJTG6cdO,https://api.spotify.com/v1/tracks/6mipiX3H43Lz...,https://api.spotify.com/v1/audio-analysis/6mip...,179998,4
117,Long Way Home,"[{'name': '5 Seconds of Summer', 'artist_link'...",46,https://i.scdn.co/image/ab67616d00001e0293432e...,https://open.spotify.com/track/7m3IIP2HPEjmtvz...,2014-06-27,album,0.455,0.880,6,...,0.2230,0.791,181.987,audio_features,7m3IIP2HPEjmtvzREOJILz,spotify:track:7m3IIP2HPEjmtvzREOJILz,https://api.spotify.com/v1/tracks/7m3IIP2HPEjm...,https://api.sp